## Spell Correction Algorithm

1. Find the misspelled word.
2. Find strings n edit distance away from the misspelled word.
3. Filter candidates using a word counter.
4. Calculate word probability.

This algorithm uses a dictionary of words to calculate the probability of a given word being correct. It generates a list of possible spelling corrections for a given word by generating all possible strings that are one or two edit distances away from the original word. It then filters this list using a word counter dictionary, and calculates the probability of each candidate word being correct. Finally, it returns the candidate word with the highest probability.

In [7]:
!pip install spacy


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [8]:
!pip install tqdm


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


### Build a word counter

In [5]:
import re
import string
import spacy
from tqdm import tqdm
import numpy as np
import pandas as pd
from collections import Counter

TypeError: issubclass() arg 1 must be a class

In [ ]:
# Make sure that the spacy english model is installed
if not spacy.util.is_package("en_core_web_sm"):
    spacy.cli.download("en_core_web_sm")
    
# Load the english model
nlp = spacy.load("en_core_web_sm", disable=["ner"])

In [ ]:
# The shakespeare.txt contains multiple lines of text, quoted with double quotes.
# Let's read each line and remove the double quotes, then append the line to a list.
read_lines = []
with open('Data/shakespeare.txt') as f:
    for line in f:
        # Regular expression to remove double quotes at beginning and end of line
        line = re.sub(r'^"|"\n$', '', line)
        read_lines.append(line)
        
# We will combine lines that end with a dot as a single sentence.
sentences = []
current_sentence = ''
for i in range(len(read_lines)):
    # Find the first index of a dot in the line. If it exists, append the line to the current sentence containing the dot, cut off the rest of the line, decrease the i counter by 1 and continue.
    # If it doesn't exist, append the line to the current sentence and continue.
    dot_index = read_lines[i].find('.')
    
    if dot_index != -1:
        # Append the line to the current sentence
        current_sentence +=  ' ' + read_lines[i][:dot_index+1]
        # Cut off the rest of the line
        read_lines[i] = read_lines[i][dot_index+1:]
        # Append the current sentence to the list of sentences
        sentences.append(current_sentence.strip())
        # Reset the current sentence
        current_sentence = ''
        # Decrease the i counter by 1 so that the current line is processed again
        i -= 1
    else:
        # Append the line to the current sentence
        current_sentence += ' ' + read_lines[i]
        
# If the current sentence is not empty, append it to the list of sentences as well
if current_sentence != '':
    sentences.append(current_sentence.strip())
    
# Let's print the first 10 sentences
for i in range(10):
    print(sentences[i])

ACT I SCENE I.
Enter KING HENRY, LORD JOHN OF LANCASTER, the EARL of WESTMORELAND, SIR WALTER BLUNT, and others So shaken as we are, so wan with care, Find we a time for frighted peace to pant, And breathe short-winded accents of new broils To be commenced in strands afar remote.
No more the thirsty entrance of this soil Shall daub her lips with her own children's blood, Nor more shall trenching war channel her fields, Nor bruise her flowerets with the armed hoofs Of hostile paces: those opposed eyes, Which, like the meteors of a troubled heaven, All of one nature, of one substance bred, Did lately meet in the intestine shock And furious close of civil butchery Shall now, in mutual well-beseeming ranks, March all one way and be no more opposed Against acquaintance, kindred and allies: The edge of war, like an ill-sheathed knife, No more shall cut his master.
As far as to the sepulchre of Christ, Whose soldier now, under whose blessed cross We are impressed and engaged to fight, Forthwi

In [ ]:
# Use spacy tokenizer to clean the text
def spacy_text_clean(text):
    """
    This function uses the spacy tokenizer to clean the text
    
    Args:
        text (str): The text to be cleaned
        
    Returns:
        tokens: A list of tokens that have been cleaned
    """
    
    # Create a spacy object
    doc = nlp(text)
    
    # Tokenize the text
    tokens = []
    for token in doc:
        if token.is_alpha:
            tokens.append(token.lower_)
    
    if len(tokens) > 1:
        return ' '.join(tokens)
    elif len(tokens) == 1:
        return tokens[0]
    else:
        ''

In [ ]:
# Initialize the token list
cleaned_sentences = []

for i in tqdm(range(len(sentences))):
    cleaned_sentences.append(spacy_text_clean(sentences[i]))

  0%|          | 0/34040 [00:00<?, ?it/s]

100%|██████████| 34040/34040 [01:46<00:00, 318.85it/s]


In [ ]:
# Remove empty strings
cleaned_sentences = [x for x in cleaned_sentences if x != '' and x is not None]

In [ ]:
# Let's print the first 10 cleaned sentences
for i in range(10):
    print(cleaned_sentences[i])

act i scene
enter king henry lord john of lancaster the earl of westmoreland sir walter blunt and others so shaken as we are so wan with care find we a time for frighted peace to pant and breathe short winded accents of new broils to be commenced in strands afar remote
no more the thirsty entrance of this soil shall daub her lips with her own children blood nor more shall trenching war channel her fields nor bruise her flowerets with the armed hoofs of hostile paces those opposed eyes which like the meteors of a troubled heaven all of one nature of one substance bred did lately meet in the intestine shock and furious close of civil butchery shall now in mutual well beseeming ranks march all one way and be no more opposed against acquaintance kindred and allies the edge of war like an ill sheathed knife no more shall cut his master
as far as to the sepulchre of christ whose soldier now under whose blessed cross we are impressed and engaged to fight forthwith a power of english shall we 

In [ ]:
def get_word_counts(sentences):
    """
    Calculates the word counts in a list of sentences
    """
    # Initialize the word counts
    word_counts = {}
    
    # Split each sentence into words, and append to a list
    words = []
    for sentence in sentences:
        words.extend(sentence.split())
    word_counts['total'] = len(words)
    
    # Count the number of times each word appears
    word_counts['counts'] = Counter(words)
    
    return word_counts

In [ ]:
# Get the word counts for the lines
word_counts = get_word_counts(cleaned_sentences)

# Print the total number of words and the top 10 most common words
print(f'Total number of words: {word_counts["total"]}')
print(f'Top 10 most common words: {word_counts["counts"].most_common(10)}')

Total number of words: 778667
Top 10 most common words: [('the', 25832), ('and', 24473), ('i', 20452), ('to', 18517), ('of', 15438), ('a', 13445), ('you', 12710), ('my', 11463), ('that', 10623), ('in', 10154)]


In [ ]:
# Create a vocabulary as a dictionary, with the words as keys and the indices as values
vocab = {word: i for i, word in enumerate(sorted(word_counts['counts'].keys()))}
vocab

{'a': 0,
 'aaron': 1,
 'abaissiez': 2,
 'abandon': 3,
 'abandoned': 4,
 'abase': 5,
 'abate': 6,
 'abated': 7,
 'abatement': 8,
 'abatements': 9,
 'abates': 10,
 'abbess': 11,
 'abbey': 12,
 'abbeys': 13,
 'abbominable': 14,
 'abbot': 15,
 'abbots': 16,
 'abbreviated': 17,
 'abed': 18,
 'abel': 19,
 'abergavenny': 20,
 'abet': 21,
 'abetting': 22,
 'abhor': 23,
 'abhorred': 24,
 'abhorring': 25,
 'abhors': 26,
 'abhorson': 27,
 'abide': 28,
 'abides': 29,
 'abilities': 30,
 'ability': 31,
 'abject': 32,
 'abjectly': 33,
 'abjects': 34,
 'abjure': 35,
 'abjured': 36,
 'able': 37,
 'abler': 38,
 'aboard': 39,
 'abode': 40,
 'aboded': 41,
 'abodements': 42,
 'aboding': 43,
 'abominable': 44,
 'abominably': 45,
 'abominations': 46,
 'abortive': 47,
 'abortives': 48,
 'abound': 49,
 'abounding': 50,
 'about': 51,
 'above': 52,
 'abraham': 53,
 'abram': 54,
 'abreast': 55,
 'abridge': 56,
 'abridged': 57,
 'abridgement': 58,
 'abridgment': 59,
 'abroach': 60,
 'abroad': 61,
 'abrogate': 62,


### Word probability

Given a word, calculate its probability using the following formula:

$$P(w_i) = \frac{C(w_i)}{M}$$
where 

$C(w_i)$ is the total number of times $w_i$ appears in the corpus.

$M$ is the total number of words in the corpus.

For example, the probability of the word 'am' in the sentence **'I am happy because I am learning'** is:

$$P(am) = \frac{C(w_i)}{M} = \frac {2}{7}$$

In [ ]:
def word_probabilities(word_counts):
    """
    Calculates the word probabilities for a dictionary of word counts
    """
    
    # Define a word_probs dictionary
    word_probs = {}
    
    # Get the total number of words
    total_words = word_counts['total']
    
    # Calculate the probability of each word
    for word, count in word_counts['counts'].items():
        word_probs[word] = count / total_words
    
    return word_probs

In [ ]:
# Get the word probabilities
word_probs = word_probabilities(word_counts)

# Print the top 10 most common words and their probabilities
print(f'Top 10 most common words: {Counter(word_probs).most_common(10)}')

Top 10 most common words: [('the', 0.03317464333277255), ('and', 0.03142935298400985), ('i', 0.026265399715154233), ('to', 0.023780383655657683), ('of', 0.019826190142898055), ('a', 0.017266687813917887), ('you', 0.016322766985116872), ('my', 0.014721312191219096), ('that', 0.013642545529732222), ('in', 0.013040234143735384)]


### String manipulation

In this part, we would like to implement 4 functions:
* `delete_letter`: given a word, it returns all the possible strings that have **one character removed**. 
* `switch_letter`: given a word, it returns all the possible strings that have **two adjacent letters switched**.
* `replace_letter`: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* `insert_letter`: given a word, it returns all the possible strings that have an **additional character inserted**. 

In [ ]:
def delete_letter(word):
    """
    Returns all possible strings obtained by deleting a letter from the word.
    """
    
    # Only delete if word has at least 1 character
    if len(word) > 1:
        # Initialize a list of words
        delete_l = []
        
        # Iterate through each character in the word
        for i in range(len(word)):
            # Delete the character at index i
            delete_l.append(word[:i] + word[i+1:])
            
        return list(set(delete_l))
    else:
        return [word]

In [ ]:
# Test the function
delete_letter('cans')

['ans', 'cas', 'can', 'cns']

In [ ]:
def switch_letter(word):
    """
    Returns all possible strings obtained by switching two consecutive characters in the word.
    Note: if the two characters are the same, we should not switch them.
    """
    # Only switch if word has at least 2 characters
    if len(word) > 1:
        # Define a list of all possible words
        switch_l = []
        
        # Loop through each character in the word
        for i in range(len(word) - 1):
            if word[i] != word[i+1]:
                switch_l.append(word[:i] + word[i+1] + word[i] + word[i+2:])
                
        return list(set(switch_l))
    else:
        return [word]

In [ ]:
# Test the function
switch_letter('eta')

['tea', 'eat']

In [ ]:
def replace_letter(word):
    """
    Returns all possible strings obtained by replacing one character in the word with another character.
    """
    # Only replace if word has at least 1 character
    if len(word) > 1:
        # Define a list of all possible words
        letters = string.ascii_lowercase
        replace_l = []
        
        # Loop through each character in the word
        for i in range(len(word)):
            for letter in letters:
                replace_l.append(word[:i] + letter + word[i+1:])
                
        return list(set(replace_l))
    else:
        return [word]

In [ ]:
# Test the function
replace_letter('can')

['cdn',
 'cad',
 'cae',
 'gan',
 'tan',
 'cau',
 'caq',
 'kan',
 'van',
 'cav',
 'cfn',
 'caa',
 'caf',
 'ean',
 'cap',
 'crn',
 'caj',
 'ccn',
 'dan',
 'ckn',
 'cmn',
 'con',
 'ctn',
 'ran',
 'man',
 'cxn',
 'cin',
 'cai',
 'cln',
 'cam',
 'yan',
 'ian',
 'cqn',
 'caw',
 'chn',
 'cay',
 'wan',
 'cao',
 'czn',
 'cvn',
 'cal',
 'xan',
 'zan',
 'caz',
 'lan',
 'car',
 'cun',
 'cbn',
 'san',
 'cax',
 'cag',
 'oan',
 'jan',
 'nan',
 'cnn',
 'han',
 'cab',
 'cas',
 'cat',
 'qan',
 'ban',
 'cgn',
 'cac',
 'cah',
 'cwn',
 'csn',
 'cjn',
 'fan',
 'cak',
 'cpn',
 'can',
 'cen',
 'uan',
 'cyn',
 'pan',
 'aan']

In [ ]:
def insert_letter(word):
    """
    Returns all possible strings obtained by inserting a new character in the word.
    """    
    # Define a list of all possible words
    letters = string.ascii_lowercase
    insert_l = []
    
    # Loop through each character in the word
    for i in range(len(word) + 1):
        for letter in letters:
            insert_l.append(word[:i] + letter + word[i:])
        
    return list(set(insert_l))

In [ ]:
# Test the function
insert_letter('at')

['yat',
 'akt',
 'aat',
 'vat',
 'atg',
 'aut',
 'nat',
 'adt',
 'apt',
 'ath',
 'atu',
 'aty',
 'zat',
 'atk',
 'dat',
 'art',
 'aft',
 'lat',
 'amt',
 'eat',
 'fat',
 'avt',
 'aqt',
 'atm',
 'iat',
 'act',
 'ast',
 'azt',
 'alt',
 'atf',
 'atz',
 'ats',
 'atj',
 'hat',
 'wat',
 'kat',
 'xat',
 'ant',
 'aot',
 'bat',
 'qat',
 'rat',
 'atn',
 'atx',
 'ati',
 'awt',
 'atl',
 'att',
 'atw',
 'cat',
 'ate',
 'atc',
 'ajt',
 'pat',
 'abt',
 'agt',
 'ait',
 'mat',
 'uat',
 'atd',
 'atp',
 'atv',
 'tat',
 'sat',
 'aet',
 'ayt',
 'ato',
 'oat',
 'aht',
 'atq',
 'gat',
 'atb',
 'axt',
 'atr',
 'ata',
 'jat']

### Edit n letters

Now, we have implemented functions to edit the strings. Let's use them to edit one or two letters.

In [ ]:
def edit_one_letter(word, allow_switches = False):
    """
    Returns a set of all possible strings obtained by applying one edit to the word.
    """
    
    edit_l = []
    
    # All all possible words obtained by deleting a letter
    edit_l.extend(delete_letter(word))
    
    # All all possible words obtained by replacing a letter
    edit_l.extend(replace_letter(word))
    
    # All all possible words obtained by inserting a letter
    edit_l.extend(insert_letter(word))
    
    # All all possible words obtained by switching two consecutive letters
    if allow_switches:
        edit_l.extend(switch_letter(word))
        
    return set(edit_l)

In [ ]:
# Test the function
edit_one_letter('at')

{'a',
 'aa',
 'aat',
 'ab',
 'abt',
 'ac',
 'act',
 'ad',
 'adt',
 'ae',
 'aet',
 'af',
 'aft',
 'ag',
 'agt',
 'ah',
 'aht',
 'ai',
 'ait',
 'aj',
 'ajt',
 'ak',
 'akt',
 'al',
 'alt',
 'am',
 'amt',
 'an',
 'ant',
 'ao',
 'aot',
 'ap',
 'apt',
 'aq',
 'aqt',
 'ar',
 'art',
 'as',
 'ast',
 'at',
 'ata',
 'atb',
 'atc',
 'atd',
 'ate',
 'atf',
 'atg',
 'ath',
 'ati',
 'atj',
 'atk',
 'atl',
 'atm',
 'atn',
 'ato',
 'atp',
 'atq',
 'atr',
 'ats',
 'att',
 'atu',
 'atv',
 'atw',
 'atx',
 'aty',
 'atz',
 'au',
 'aut',
 'av',
 'avt',
 'aw',
 'awt',
 'ax',
 'axt',
 'ay',
 'ayt',
 'az',
 'azt',
 'bat',
 'bt',
 'cat',
 'ct',
 'dat',
 'dt',
 'eat',
 'et',
 'fat',
 'ft',
 'gat',
 'gt',
 'hat',
 'ht',
 'iat',
 'it',
 'jat',
 'jt',
 'kat',
 'kt',
 'lat',
 'lt',
 'mat',
 'mt',
 'nat',
 'nt',
 'oat',
 'ot',
 'pat',
 'pt',
 'qat',
 'qt',
 'rat',
 'rt',
 'sat',
 'st',
 't',
 'tat',
 'tt',
 'uat',
 'ut',
 'vat',
 'vt',
 'wat',
 'wt',
 'xat',
 'xt',
 'yat',
 'yt',
 'zat',
 'zt'}

In [ ]:
def edit_two_letters(word, allow_switches = False):
    """
    Returns all possible strings obtained by editing the word twice.
    """
    
    # Find all possible words obtained by editing the word once
    edit_l = edit_one_letter(word, allow_switches)
    
    # Define a set of all possible words obtained by editing the word twice
    edit_2_set = set()
    
    # Add all possible words obtained by editing the word once, to the set of words obtained by editing the word twice
    for edit_word in edit_l:
        edit_2_set.update(edit_one_letter(edit_word, allow_switches))
        
    return edit_2_set

In [ ]:
edit_2 = edit_two_letters('at')
print(f'Number of strings obtained by editing the word twice: {len(edit_2)}')

Number of strings obtained by editing the word twice: 7129


### Suggest spelling correction

Given the 2 distance edit candidates, find the most probable word for suggestion.

In [ ]:
def get_corrections(word, vocab, word_probs, k=2):
    """
    Returns the k most probable words for spelling correction.
    
    Args:
        word: The word to be corrected.
        vocab: A dictionary containing all the words in the vocabulary.
        word_probs: A dictionary containing the word probabilities.
        k: The number of most probable words to return.
        
    Returns: a list of tuples of the form (word, probability).
    """
    
    # Get the set of all possible words obtained by editing the word once
    edit_one = edit_one_letter(word).intersection(vocab.keys())
    
    # Get the set of all possible words obtained by editing the word twice
    edit_two = edit_two_letters(word).intersection(vocab.keys())
    
    # Define a set of all possible words
    suggestions = edit_one.union(edit_two)
    
    # Also include the word itself if it is in the dictionary
    if word in vocab:
        suggestions.add(word)
        
    # Suggestions with their probabilities
    suggestions_prob = {word: word_probs[word] for word in suggestions}
    
    # Get the k most probable suggestions
    best_words = Counter(suggestions_prob).most_common(k)
    
    return best_words

In [ ]:
# Test the function
get_corrections('dys', vocab, word_probs, k=10)

### Minimum edit distance

Now we have implemented the simple autocorrect functions. How do we know how many steps it takes to modify a word from 'waht' to 'what'?

In this part, we are implementing the minimum edit distance algorithm by applying dynamic programming. This breaks a problem down into subproblems which can be combined to form the final solution.

In [ ]:
def min_edit_distance(source, target, ins_cost=1, del_cost=1, rep_cost=2):
    '''
    Input: 
        source: a string corresponding to the string you are starting with
        target: a string corresponding to the string you want to end with
        ins_cost: an integer setting the insert cost
        del_cost: an integer setting the delete cost
        rep_cost: an integer setting the replace cost
    Output:
        D: a matrix of len(source)+1 by len(target)+1 containing minimum edit distances
        med: the minimum edit distance (med) required to convert the source string to the target
    '''
    # use deletion and insert cost as  1
    m = len(source)
    n = len(target)
    #initialize cost matrix with zeros and dimensions (m+1,n+1) 
    D = np.zeros((m + 1, n + 1), dtype=int)

    # Fill in column 0, from row 1 to row m, both inclusive
    for row in range(1, m + 1):  # Replace None with the proper range
        D[row, 0] = D[row - 1, 0] + del_cost

    # Fill in row 0, for all columns from 1 to n, both inclusive
    for col in range(1, n + 1):  # Replace None with the proper range
        D[0, col] = D[0, col - 1] + ins_cost

    # Loop through row 1 to row m, both inclusive
    for row in range(1, m + 1):

        # Loop through column 1 to column n, both inclusive
        for col in range(1, n + 1):

            # Intialize r_cost to the 'replace' cost that is passed into this function
            r_cost = rep_cost

            # Check to see if source character at the previous row
            # matches the target character at the previous column, 
            if source[row - 1] == target[col - 1]:  # Replace None with a proper comparison
                # Update the replacement cost to 0 if source and target are the same
                r_cost = 0

            # Update the cost at row, col based on previous entries in the cost matrix
            # Refer to the equation calculate for D[i,j] (the minimum of three calculated costs)
            D[row, col] = min([D[row - 1, col] + del_cost, D[row, col - 1] + ins_cost, D[row - 1, col - 1] + r_cost])

    # Set the minimum edit distance with the cost found at row m, column n 
    med = D[m, n]

    return D, med

In [ ]:
source = 'play'
target = 'stay'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ", min_edits, "\n")
idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns=cols)
print(df)